# Installation and Repository cloning

In [1]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
#!pip install adversarial-robustness-toolbox -U
#!pip install git+https://github.com/RobustBench/robustbench.git
#!pip install multiprocess
#!pip install matplotlib
#!pip install pillow
#!pip install foolbox
#!pip install numba

## Import Libraries

In [2]:
import torch
import utils
import adversarial_attack.attack_utils as attack_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/shao/anaconda3/envs/art/lib/python3.12/site-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


## Load the dataset

In [3]:
dataset = 'imagenet' #'cifar10' #'imagenet'
splitsize = 100 # full, int: splitsize
xtest, ytest = utils.load_dataset(dataset= dataset, dataset_split=splitsize, root='../datasets')

## Load and Prepare Adversarial Model

In [4]:
model = 'standard_resnet50' # CIFAR10: (MainiAVG, MainiMSD, CroceL1, corruption_robust, standard), IN: (standard_resnet50, Wang2023Better_WRN-28-10, Wang2023Better_WRN-28-10, Salman2020Do_R18)
net, art_net, fb_net, alias = utils.get_model(dataset= dataset, modelname=model,norm='L2') # specify the norm in case of robustbench models

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/shao/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:08<00:00, 11.9MB/s]


## Test Accuracy

In [5]:
utils.test_accuracy(net, xtest, ytest)
#print(ytest)


Accuracy of the test set is: 82.000%



## Parameters 

In [6]:
Experiment = attack_utils.Experiment_class(art_net, fb_net, net, xtest, ytest, alias,
                        epsilon_l1 = 12, 
                        epsilon_l2= 0.5, 
                        eps_iter = 0.1,  
                        norm = 1,  # 1, 2, np.inf
                        max_iterations =300, 
                        verbose=False)

# Hyperparameter Sweep Experiment

In [ ]:
#hyperparameter sweep. Pick only one model.
attack_type = 'exp_attack_l1'
hyperparameter_range=[1.0]
hyperparameter = 'learning_rate' #'beta'

results_dict_hyperparameter_sweep = Experiment.hyperparameter_sweep(hyperparameter=hyperparameter, range=hyperparameter_range, attack_type=attack_type)

In [ ]:
#Attack comparison
attack_types = [#'fast_gradient_method',
                #'projected_gradient_descent',
                #'pgd_early_stopping',
                #'deep_fool',
                #'brendel_bethge',
                #'carlini_wagner_l2',
                #'elastic_net',
                #'exp_attack',
                #'exp_attack_smooth',
                #'exp_attack_l1_l2',
                #'auto_projected_gradient_descent',
                #'elastic_net_L1_rule',
                #'elastic_net_L1_rule_higher_beta',
                #'ART_AutoAttack',
                #'original_AutoAttack',
                'exp_attack_l1',
                'custom_apgd'
                ]

results_dict_attack_comparison = Experiment.attack_comparison(attack_types)